<a href="https://colab.research.google.com/github/dan-manolescu/data-structures-fun/blob/main/C9_Hash_Tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from typing import Any

Utility function to generate an integer hash value for the given key. Currently only supports integer and string types for a key. For other types we used a generic Python implementation for hash.

In [13]:
def HashFunction(key: Any, size: int) -> int:
    if type(key) is int:
        return IntHash(key, size)
    elif type(key) is str:
        return StringHash(key, size)
    else:
        return hash(key) % size

def IntHash(key: int, size: int) -> int:
    return key % size

def StringHash(key: str, size: int) -> int:
    '''
    Horner's method of hashing strings.
    Instead of adding the letters values we multiply the running sum
    of letters by a constant after each addition.
    '''
    # CONST is a prime number larger than the largest value for any character.
    # The highest Unicode character code value is U+10FFFF, which is equal to 1,114,111
    CONST = 1114117
    total = 0
    for character in key:
        total = CONST * total + ord(character)
    return total % size

# Chaining

Chaining approach for handling collisions within the hash table (via the usage of a linked list for every entry in the hash table).

In [14]:
class ListNode:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None

In [15]:
class HashTable:
    def __init__(self, size: int):
        self.size = size
        self.bins = [None] * size

    def HashTableInsert(self, key: Any, value: Any) -> None:
        # Compute the hash value for the key.
        hash_value = HashFunction(key, self.size)

        # If the bin is empty, create a new linked list.
        if self.bins[hash_value] == None:
            self.bins[hash_value] = ListNode(key, value)
            return

        # Check if the key exists already in the table.
        current = self.bins[hash_value]
        while current.key != key and current.next != None:
            current = current.next
        # If the key is found then update the value, otherwise append the new key at the end of the list.
        if current.key == key:
            current.value = value
        else:
            current.next = ListNode(key, value)
        return

    def HashTableLookup(self, key: Any) -> Any:
        # Compute the hash value for the key.
        hash_value = HashFunction(key, self.size)
        # Check if the corresponding bin is empty.
        if self.bins[hash_value] == None:
            return None

        # Scan over each element of the list and return the value for the matching key.
        current = self.bins[hash_value]
        while current.key != key and current.next != None:
            current = current.next
        if current.key == key:
            return current.value
        # We made it to the end without finding a matching key so return None.
        return None

    def HashTableRemove(self, key: Any) -> ListNode:
        # Compute the hash value for the key and check the corresponding bin if it's empty.
        hash_value = HashFunction(key, self.size)
        if self.bins[hash_value] == None:
            return None

        # Scan through the linked list for the matching key and
        # track the final linked list node before the current node
        current = self.bins[hash_value]
        last = None
        while current.key != key and current.next != None:
            last = current
            current = current.next
        if current.key == key:
            if last != None:
                # Remove the current node by simply modifying the last node's
                # next pointer to skip the node we are removing
                last.next = current.next
            else:
                # We are removing the first element in the list.
                self.bins[hash_value] = current.next
            return current
        return None


Some examples

In [18]:
ht = HashTable(5)
ht.HashTableInsert(5, 'Value for 5')
ht.HashTableInsert(20, 'Value for 20')
ht.HashTableInsert(21, 'Value for 21')
ht.HashTableInsert(34, 'Value for 14')
ht.HashTableInsert(41, 'Value for 41')
print(ht.HashTableLookup(20))
print(ht.HashTableLookup(25))

Value for 20
None


# Linear Probing

In [16]:
class HashTableEntry:
    def __init__(self, key, value):
        self.key = key
        self.value = value

The class that implements a Hash Table with Linear Probing needs to keep the number of keys currently stored in the table. If the number of keys reaches the size of the table then there are no more available bins.

For a production environment we can think of a class able to dynamically increase its size when it starts to be full.

In [17]:
class HashTableLP:
    def __init__(self, size: int):
        self.size = size
        self.num_keys = 0
        self.bins = [None] * size

    def HashTableInsert(self, key: Any, value: Any) -> bool:
        # Compute the key hash value.
        index = HashFunction(key, self.size)
        # Maintain a count of bins to check to prevent an infinite loop if the table is full.
        count = 0

        current = self.bins[index]
        # Loop through each bin and test if:
        # we found an empty bin
        # we found a dummy entry (an entry with key=None added when removing)
        # we found the target key
        # and if we searched all the bins.
        while current != None and current.key != None and current.key != key and count != self.size:
            index += 1
            # After incrementing the index check if the index should wrap back to the beginning of the table.
            if index >= self.size:
                index = 0
            current = self.bins[index]
            count += 1

        # After the loop is complete if we have examinged every bin in the table
        # without finding the key then the table is full.
        if count == self.size:
            return False

        # Check if we found an empty or dummy bin (create a new entry)
        # or a matching key (update the value).
        if current == None or current.key == None:
            self.bins[index] = HashTableEntry(key, value)
            self.num_keys += 1
        else:
            self.bins[index].value = value
        return True

    def HashTableLookup(self, key: Any) -> Any:
        # Compute the hash value for the key to get the starting location for the search.
        index = HashFunction(key, self.size)
        # Maintain a count of bins to prevent an infinite loop if the table is full.
        count = 0

        current = self.bins[index]
        # Loop through the table and test these conditions:
        # we found an empty bin
        # we found the target key
        # we've searched all the bins.
        while current != None and current.key != key and count != self.size:
            index += 1
            # If the search has run off the end of the table wrap it back at the start.
            if index >= self.size:
                index = 0
            current = self.bins[index]
            count += 1

        # Return the value if we found a match.
        if current != None and current.key == key:
            return current.value
        return None

    def HashTableRemove(self, key) -> HashTableEntry:
        # Method chosen here is to insert dummy values into the bin for the
        # removed entry. This ensures that entries added afterward due to collisions
        # are still accessible.
        # One other potential solution would be to scan through the table
        # and fix later entries.

        index = HashFunction(key, self.size)
        count = 0

        current = self.bins[index]
        while current != None and current.key != key and count != self.size:
            index += 1
            if index >= self.size:
                index = 0
            current = self.bins[index]
            count += 1

        # If we found the entry with the matching key then replace it with a dummy entry.
        if current != None and current.key == key:
            self.bins[index] = HashTableEntry(None, None)
            self.num_keys -= 1
            return current
        return None

Some examples

In [20]:
htl = HashTableLP(5)
print(htl.HashTableInsert(5, 'Value for 5'))
print(htl.HashTableInsert(20, 'Value for 20'))
print(htl.HashTableInsert(21, 'Value for 21'))
print(htl.HashTableInsert(34, 'Value for 14'))
print(htl.HashTableInsert(41, 'Value for 41'))
print(htl.HashTableInsert(25, 'Value for 25'))

True
True
True
True
True
False


In [21]:
htl.HashTableLookup(20)

'Value for 20'

In [23]:
htl.HashTableRemove(20)
print(htl.HashTableInsert(25, 'Value for 25'))

True
